In [2]:
import numpy as np
import pandas as pd
from os import listdir
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from metrics_calculation import metrics

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_dir = '../CMF_FFS/data/processed/'

## Searching for feature importance

In [26]:
y_list = []
pred_list = []
features_list = []
coef_list = []
for i in listdir(data_dir):
    data = pd.read_csv(data_dir + i)
    data['Equity Charge'] = [i for i in range(len(data['Equity Charge']))]
    y = data['Revenue']
    data.drop(columns=['Revenue'], inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(data, y, train_size=38, shuffle=False)
    
    model = Lasso()

    model.fit(x_train, y_train)
    coef_list.append(model.coef_)
    predictions = model.predict(x_test)
    y_list.append(y_test)
    pred_list.append(predictions)
    features_list.append(len(x_test.columns))
metrics(y_list, pred_list, features_list, one_model=False)

Mean MAPE = 0.48520016269903776
Mean WAPE = 0.4831421455045152
Mean MSE = 9631563.724151878
Mean R2_adj 6.644024450330613


## Getting top 10 features

In [46]:
features = dict.fromkeys(range(73),0)
for i in coef_list:
    for j in np.where(i>0)[0]:
        features[j] += 1

In [51]:
feature_importance = {k: v for k, v in sorted(features.items(), key=lambda item: item[1])}

In [94]:
ind = list(feature_importance.keys())[-10:]

## Fitting with top 10 features

In [96]:
y_list = []
pred_list = []
features_list = []
for i in listdir(data_dir):
    data = pd.read_csv(data_dir + i)
    data['Equity Charge'] = [i for i in range(len(data['Equity Charge']))]
    y = data['Revenue']
    data.drop(columns=['Revenue'], inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(data, y, train_size=38, shuffle=False)
    
    x_train = x_train.iloc[:, ind]
    x_test = x_test.iloc[:, ind]
    
    model = Lasso()

    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    y_list.append(y_test)
    pred_list.append(predictions)
    features_list.append(len(x_test.columns))
metrics(y_list, pred_list, features_list, one_model=False)

Mean MAPE = 0.1977819346298819
Mean WAPE = 0.19342538901126866
Mean MSE = 68719.20589049453
Mean R2_adj 2.351410209428366
